# RESNET50 Image Classification Model Acceleration

In [30]:
%pip install torch torchvision 
%pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 55.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 145.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.4/315.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.5/934.5 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.

In [31]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.io import decode_image 

from torchvision.models import resnet50, ResNet50_Weights 

import torch
import os
import time

### check whether GPU is available or not: 

In [32]:
print("GPU is available" if torch.cuda.is_available() else "GPU is not available")

GPU is available


### define the dataset class for our inference images:

In [33]:
class InferenceDataset(Dataset):
    def __init__(self, images_dir: str, transform = None): 
        self.images_dir = images_dir
        self.transform = transform
        self.images = sorted([img for img in os.listdir(self.images_dir) 
                              if img.lower().endswith((".jpg", ".jpeg", ".png"))
                              ])
    
    def __getitem__(self, idx): 
        img_path = os.path.join(self.images_dir, self.images[idx])
        img = decode_image(img_path)
        
        if self.transform: 
            img = self.transform(img).unsqueeze(0)
        
        return img

### define the dataloader

In [34]:
def data_loader(dataset, batch_size, num_workers, pin_memory):
    return DataLoader(
        dataset,
        batch_size=batch_size,        
        shuffle=False, # no shuffling to keep the order of images processing identical for fair comparison
        num_workers=num_workers,        
        pin_memory=pin_memory   
        )

### define `gpu_inference` function for GPU

In [35]:
def gpu_inference(model, loader, non_blocking = True): 
    device = torch.device('cuda')
    #move the model to gpu 
    model = model.to(device)
    #set to evaluation mode, this disables dropouts and batch normalizations
    model.eval()
     
    torch.cuda.synchronize()
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    start.record()

    with torch.no_grad():
        for x in loader:
            x = x.to(device, non_blocking=non_blocking)
            _ = model(x)

    end.record()
    torch.cuda.synchronize()

    gpu_time = start.elapsed_time(end)
    print("GPU time (ms):", gpu_time)
    return gpu_time



### define `cpu_inference` function for CPU

In [36]:
def cpu_inference(model, loader): 
    device = torch.device("cpu")
    model = model.to(device)
    model.eval()

    start = time.time()
    with torch.no_grad():
        for x in loader:
            _ = model(x)

    end = time.time()

    cpu_time = end - start
    print("CPU time (s):", cpu_time)
    return cpu_time


### import the ResNet50 model with default (best availabe) weights 

In [37]:
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)

### weights come with the preprocessing pipeline matching the characteristics of the training data:

In [38]:
preprocess = weights.transforms()

### here comes the fun: 